# Power BI Semantic Model Connection (Local)

Connect to Power BI Semantic Models from a **local Python environment** using the REST API.

## Prerequisites
1. **Azure CLI authentication**: Run `az login` before starting
2. **Power BI Premium, PPU, or Fabric capacity**: Required for executing DAX queries
3. **Workspace access**: You need at least read access to the workspace and semantic model

> **Note**: The `semantic-link` library's XMLA-based functions (`read_table`, `list_tables`) only work inside Fabric notebooks. This notebook uses the REST API which works locally.

In [ ]:
import pandas as pd
import requests
import json
from azure.identity import DefaultAzureCredential, DeviceCodeCredential
from azure.core.exceptions import ClientAuthenticationError

In [ ]:
# Authenticate to Power BI
try:
    credential = DefaultAzureCredential()
    token = credential.get_token("https://analysis.windows.net/powerbi/api/.default")
    print("✓ Authenticated using DefaultAzureCredential")
except ClientAuthenticationError:
    print("DefaultAzureCredential failed. Using DeviceCodeCredential...")
    credential = DeviceCodeCredential()
    token = credential.get_token("https://analysis.windows.net/powerbi/api/.default")
    print("✓ Authenticated using DeviceCodeCredential")

## Power BI REST API Helper Class

A reusable class for interacting with Power BI semantic models from local Python.

In [ ]:
class PowerBIClient:
    """Client for interacting with Power BI semantic models via REST API."""
    
    BASE_URL = "https://api.powerbi.com/v1.0/myorg"
    
    def __init__(self, credential):
        self.credential = credential
        self._workspaces_cache = None
    
    def _get_headers(self):
        token = self.credential.get_token("https://analysis.windows.net/powerbi/api/.default")
        return {
            "Authorization": f"Bearer {token.token}",
            "Content-Type": "application/json"
        }
    
    def list_workspaces(self) -> pd.DataFrame:
        """List all workspaces the user has access to."""
        response = requests.get(f"{self.BASE_URL}/groups", headers=self._get_headers())
        response.raise_for_status()
        workspaces = response.json().get("value", [])
        self._workspaces_cache = {ws["name"]: ws for ws in workspaces}
        return pd.DataFrame(workspaces)
    
    def get_workspace_id(self, workspace_name: str) -> str:
        """Get workspace ID by name."""
        if not self._workspaces_cache:
            self.list_workspaces()
        ws = self._workspaces_cache.get(workspace_name)
        if not ws:
            raise ValueError(f"Workspace '{workspace_name}' not found")
        return ws["id"]
    
    def is_premium(self, workspace_name: str) -> bool:
        """Check if workspace is on Premium/Fabric capacity."""
        if not self._workspaces_cache:
            self.list_workspaces()
        ws = self._workspaces_cache.get(workspace_name)
        return ws.get("isOnDedicatedCapacity", False) if ws else False
    
    def list_datasets(self, workspace_name: str) -> pd.DataFrame:
        """List all datasets in a workspace."""
        workspace_id = self.get_workspace_id(workspace_name)
        url = f"{self.BASE_URL}/groups/{workspace_id}/datasets"
        response = requests.get(url, headers=self._get_headers())
        response.raise_for_status()
        return pd.DataFrame(response.json().get("value", []))
    
    def get_dataset_id(self, workspace_name: str, dataset_name: str) -> str:
        """Get dataset ID by name."""
        df = self.list_datasets(workspace_name)
        match = df[df["name"] == dataset_name]
        if match.empty:
            raise ValueError(f"Dataset '{dataset_name}' not found in workspace '{workspace_name}'")
        return match.iloc[0]["id"]
    
    def execute_dax(self, workspace_name: str, dataset_name: str, dax_query: str) -> pd.DataFrame:
        """Execute a DAX query and return results as DataFrame."""
        workspace_id = self.get_workspace_id(workspace_name)
        dataset_id = self.get_dataset_id(workspace_name, dataset_name)
        
        url = f"{self.BASE_URL}/groups/{workspace_id}/datasets/{dataset_id}/executeQueries"
        payload = {
            "queries": [{"query": dax_query}],
            "serializerSettings": {"includeNulls": True}
        }
        
        response = requests.post(url, headers=self._get_headers(), json=payload)
        
        if response.status_code != 200:
            error = response.json().get("error", {})
            raise RuntimeError(f"DAX query failed: {error.get('message', response.text)}")
        
        result = response.json()
        tables = result.get("results", [{}])[0].get("tables", [])
        if tables:
            rows = tables[0].get("rows", [])
            return pd.DataFrame(rows)
        return pd.DataFrame()
    
    def read_table(self, workspace_name: str, dataset_name: str, table_name: str, 
                   top_n: int = None, columns: list = None) -> pd.DataFrame:
        """Read data from a table in a semantic model."""
        col_list = ", ".join(columns) if columns else "*"
        
        if top_n:
            dax = f"EVALUATE TOPN({top_n}, '{table_name}')"
        else:
            dax = f"EVALUATE '{table_name}'"
        
        return self.execute_dax(workspace_name, dataset_name, dax)
    
    def evaluate_measure(self, workspace_name: str, dataset_name: str, 
                         measure: str, group_by: list = None) -> pd.DataFrame:
        """Evaluate a measure, optionally grouped by columns."""
        if group_by:
            cols = ", ".join(group_by)
            dax = f"""
            EVALUATE 
            SUMMARIZECOLUMNS(
                {cols},
                "Result", {measure}
            )
            """
        else:
            dax = f"EVALUATE ROW(\"Result\", {measure})"
        
        return self.execute_dax(workspace_name, dataset_name, dax)

# Initialize the client
pbi = PowerBIClient(credential)
print("✓ PowerBIClient initialized")

## 1. List Workspaces and Datasets

In [ ]:
# List workspaces
df_workspaces = pbi.list_workspaces()
print(f"Found {len(df_workspaces)} workspaces")
display(df_workspaces[["name", "id", "isOnDedicatedCapacity"]].head(10))

In [ ]:
# Configuration - set your workspace and dataset names
WORKSPACE_NAME = "perry-pbi-demo-workspace"
DATASET_NAME = "Customer Profitability Sample"

# Check if workspace is on Premium capacity
if pbi.is_premium(WORKSPACE_NAME):
    print(f"✓ Workspace '{WORKSPACE_NAME}' is on Premium/Fabric capacity")
else:
    print(f"✗ Workspace '{WORKSPACE_NAME}' is NOT on Premium capacity - DAX queries will fail!")

# List datasets in workspace
df_datasets = pbi.list_datasets(WORKSPACE_NAME)
print(f"\nDatasets in '{WORKSPACE_NAME}':")
display(df_datasets[["name", "id", "configuredBy"]])

## 2. Read Data from a Table

In [ ]:
# Read the first 10 rows from the 'Customer' table
df_customers = pbi.read_table(WORKSPACE_NAME, DATASET_NAME, "Customer", top_n=10)
print(f"Retrieved {len(df_customers)} rows from 'Customer' table")
display(df_customers)

## 3. Execute Custom DAX Query

In [ ]:
# Execute a custom DAX query
dax_query = """
EVALUATE
SUMMARIZECOLUMNS(
    'Customer'[State],
    "Customer Count", COUNTROWS('Customer')
)
ORDER BY [Customer Count] DESC
"""

df_result = pbi.execute_dax(WORKSPACE_NAME, DATASET_NAME, dax_query)
print(f"Query returned {len(df_result)} rows")
display(df_result)

## 4. Evaluate a Measure

In [ ]:
# Evaluate a measure (replace with an actual measure from your model)
# Example: Evaluate [Total Revenue] grouped by State
try:
    df_measure = pbi.evaluate_measure(
        WORKSPACE_NAME, 
        DATASET_NAME, 
        "[Total Revenue]",  # Replace with your measure name
        group_by=["'Customer'[State]"]
    )
    display(df_measure.head(10))
except Exception as e:
    print(f"Note: {e}")
    print("Tip: Replace '[Total Revenue]' with an actual measure from your semantic model")